In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [0]:
import cv2
import imutils
from google.colab.patches import cv2_imshow

In [0]:
data = pd.read_csv('/content/drive/My Drive/Facial Expressions/fer2013/fer2013.csv')
data.tail()

,emotion,pixels,Usage
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest
35886,2,19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...,PrivateTest


In [0]:
emotions = ['Angry','Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [0]:
pri_test = data[data['Usage']=='PrivateTest']
print(pri_test.shape)
pri_test.head()

(3589, 3)


,emotion,pixels,Usage
32298,0,170 118 101 88 88 75 78 82 66 74 68 59 63 64 6...,PrivateTest
32299,5,7 5 8 6 7 3 2 6 5 4 4 5 7 5 5 5 6 7 7 7 10 10 ...,PrivateTest
32300,6,232 240 241 239 237 235 246 117 24 24 22 13 12...,PrivateTest
32301,4,200 197 149 139 156 89 111 58 62 95 113 117 11...,PrivateTest
32302,2,40 28 33 56 45 33 31 78 152 194 200 186 196 20...,PrivateTest


In [0]:
pri_labels = pri_test['emotion']
pri_labels.head()

32298    0
32299    5
32300    6
32301    4
32302    2
Name: emotion, dtype: int64

In [0]:
train = data.iloc[:32298]
print(train.shape)
train.tail()

(32298, 3)


,emotion,pixels,Usage
32293,4,178 176 172 173 173 174 176 173 166 166 206 22...,PublicTest
32294,3,25 34 42 44 42 47 57 59 59 58 54 51 50 56 63 6...,PublicTest
32295,4,255 255 255 255 255 255 255 255 255 255 255 25...,PublicTest
32296,4,33 25 31 36 36 42 69 103 132 163 175 183 187 1...,PublicTest
32297,4,61 63 59 75 151 159 166 161 143 170 127 131 18...,PublicTest


In [0]:
train_labels = train['emotion']
train_labels.tail()

32293    4
32294    3
32295    4
32296    4
32297    4
Name: emotion, dtype: int64

In [0]:
train_pixels=[]

In [0]:
for i in range(len(train)):
  x = list((train['pixels'][i].split(" ")))
  x = np.asarray(x, dtype=np.int32)
  x = np.reshape(np.ravel(x), (48, 48,1))
  train_pixels.append(x)

In [0]:
train_pixels[0].shape

(48, 48, 1)

In [0]:
X_train = np.asarray(train_pixels)
X_train.shape

(32298, 48, 48, 1)

In [0]:
pri_test_pixels=[]
for i in range(32298,len(pri_test)+32298):
  x = list((pri_test['pixels'][i].split(" ")))
  x = np.asarray(x, dtype=np.int32)
  x = np.reshape(np.ravel(x), (48, 48,1))
  pri_test_pixels.append(x)

In [0]:
X_test_pri = np.asarray(pri_test_pixels)
X_test_pri.shape

(3589, 48, 48, 1)

In [0]:
X_train[0].shape[0]

48

In [0]:
X_train = X_train/255.0
X_test_pri = X_test_pri/255.0

In [0]:
train_labels.shape

(32298,)

In [0]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
pri_labels = to_categorical(pri_labels)

In [0]:
train_labels[0].shape

(7,)

In [0]:
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
import keras
from keras.layers import Dropout
from keras.regularizers import l2

In [0]:
def cnn(input_shape, classes, l2_reg=0.01):
    reg = l2(l2_reg)

    
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
               use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = (Dropout(0.7))(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
               use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)
    x = (Dropout(0.7))(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

     
    x = (Dropout(0.7))(x)
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

     
    x = (Dropout(0.7))(x)
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

     
    x = (Dropout(0.7))(x)
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
               
               padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax', name='predictions')(x)

    model = Model(img_input, output)
    return model

In [0]:
model = cnn(X_train.shape[1:],len(emotions),0.001)

In [0]:
adam = keras.optimizers.Adam(0.001)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [0]:
model.fit(X_train, train_labels, epochs=100, batch_size=256)

Epoch 1/100
32298/32298 [==============================] - 31s 969us/step - loss: 1.8833 - acc: 0.2179
Epoch 2/100
32298/32298 [==============================] - 25s 759us/step - loss: 1.8372 - acc: 0.2330
Epoch 3/100
32298/32298 [==============================] - 25s 759us/step - loss: 1.8170 - acc: 0.2413
Epoch 4/100
32298/32298 [==============================] - 25s 763us/step - loss: 1.8071 - acc: 0.2465
Epoch 5/100
32298/32298 [==============================] - 25s 761us/step - loss: 1.8013 - acc: 0.2498
Epoch 6/100
32298/32298 [==============================] - 25s 762us/step - loss: 1.7937 - acc: 0.2531
Epoch 7/100
32298/32298 [==============================] - 25s 759us/step - loss: 1.7865 - acc: 0.2607
Epoch 8/100
32298/32298 [==============================] - 25s 759us/step - loss: 1.7787 - acc: 0.2647
Epoch 9/100
32298/32298 [==============================] - 25s 759us/step - loss: 1.7665 - acc: 0.2787
Epoch 10/100
32298/32298 [==============================] - 24s 758us/ste

In [0]:
model.save("Emotion_1.h5")

In [0]:
from keras.models import load_model

In [0]:
trained_model = load_model('/content/Emotion_1.h5')

trained_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_84 (Conv2D)              (None, 46, 46, 8)    72          input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_80 (BatchNo (None, 46, 46, 8)    32          conv2d_84[0][0]                  
__________________________________________________________________________________________________
activation_35 (Activation)      (None, 46, 46, 8)    0           batch_normalization_80[0][0]     
____________________________________________________________________________________________

In [0]:
pri_pred_load = trained_model.predict(X_test_pri,batch_size=128)

In [0]:
pri_pred = model.predict(X_test_pri,batch_size=128)

In [0]:
emotions[np.argmax(pri_pred[0])],emotions[np.argmax(pri_pred_load[0])]

('Disgust', 'Disgust')

In [0]:
train_pred = model.predict(X_train,batch_size=128)

In [0]:
total =0
pred = 0
for i in range(len(train_pred)):
  y_cap = emotions[np.argmax(train_pred[i])]
  y = emotions[np.argmax(train_labels[i])]
  if(y==y_cap):
    pred = pred + 1
  total = total + 1
print("Training Test Accuracy:",pred/total*100)

Training Test Accuracy: 56.41525791070654


In [0]:
total =0
pred = 0
for i in range(len(pri_pred)):
  y_cap = emotions[np.argmax(pri_pred[i])]
  y = emotions[np.argmax(pri_labels[i])]
  if(y==y_cap):
    pred = pred + 1
  total = total + 1
print("Private Test Accuracy:",pred/total*100)

Private Test Accuracy: 53.246029534689335
